In [ ]:
import os, json, numpy as np, pandas as pd, polars as pl, joblib, torch
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import kaggle_evaluation.cmi_inference_server

device = "cuda" if torch.cuda.is_available() else "cpu"
COMP_DIR = "/kaggle/input/cmi-detect-behavior-with-sensor-data"
ART_DIR = "/kaggle/input/model_cmi"

In [ ]:
def pad_to_len(X, pad_len, pad_mode="post", trunc_mode="post", value=0.0):
    X = np.asarray(X, dtype=np.float32); T, C = X.shape
    if T == pad_len: 
        return X.copy()
    if T > pad_len:
        if trunc_mode == "post": out = X[:pad_len]
        elif trunc_mode == "pre": out = X[-pad_len:]
        else: start = (T - pad_len)//2; out = X[start:start+pad_len]
        return out.astype(np.float32, copy=False)
    
    pad_total = pad_len - T
    if pad_mode == "post": 
        pb, pa = 0, pad_total
    elif pad_mode == "pre": 
        pb, pa = pad_total, 0
    else: 
        pb = pad_total//2
        pa = pad_total - pb
        
    return np.pad(X, ((pb,pa),(0,0)), constant_values=value).astype(np.float32, copy=False)

In [ ]:
class AttnPool(nn.Module):
    def __init__(self, dim, dropout=0.1):
        super().__init__() 
        self.proj = nn.Linear(dim, dim)
        self.v = nn.Linear(dim,1,bias=False) 
        self.drop = nn.Dropout(dropout)
        
    def forward(self, H, mask=None):
        e = self.v(torch.tanh(self.proj(H))).squeeze(-1)
        if mask is not None: 
            e = e.masked_fill(~mask, float("-inf"))
        w = torch.softmax(e, dim=1).unsqueeze(-1)
        
        return self.drop((H*w).sum(1)), w.squeeze(-1)

class RNNWithAttn(nn.Module):
    def __init__(self, input_dim, num_classes, rnn_type="lstm", hidden_size=128, num_layers=2, bidirectional=True, input_dropout=0.1, attn_dropout=0.1, fc_dropout=0.2):
        super().__init__()
        self.in_drop = nn.Dropout(input_dropout)
        rnn = nn.LSTM if rnn_type.lower() == "lstm" else nn.GRU
        self.rnn = rnn(input_dim, hidden_size, num_layers, batch_first=True, bidirectional=bidirectional, dropout=0.0 if num_layers==1 else 0.1)
        d = hidden_size*(2 if bidirectional else 1)
        self.attn = AttnPool(d, dropout=attn_dropout)
        self.head = nn.Sequential(nn.Linear(d,d), nn.ReLU(), nn.Dropout(fc_dropout), nn.Linear(d, num_classes))
        
    def forward(self, x_pad, lengths):
        x = self.in_drop(x_pad)
        packed = pack_padded_sequence(x, lengths.cpu(), batch_first=True, enforce_sorted=False)
        packed_out, _ = self.rnn(packed)
        H, _ = pad_packed_sequence(packed_out, batch_first=True, total_length=x_pad.size(1))
        idx = torch.arange(x_pad.size(1), device=x_pad.device).unsqueeze(0)
        mask = idx < lengths.unsqueeze(1)
        ctx, _ = self.attn(H, mask=mask)
        return self.head(ctx)

In [ ]:
with open(os.path.join(ART_DIR,"meta.json")) as f: meta = json.load(f)
with open(os.path.join(ART_DIR,"id2label.json")) as f: id2label = {int(k):v for k,v in json.load(f).items()}
scaler = joblib.load(os.path.join(ART_DIR,"scaler.pkl"))
feat_cols = meta["feat_cols"]; pad_len = int(meta["pad_len"])

model = RNNWithAttn(input_dim=len(feat_cols), num_classes=len(id2label))
state = torch.load(os.path.join(ART_DIR,"model_state_dict.pt"), map_location=device)
model.load_state_dict(state); model.to(device).eval()

In [ ]:
def _prep_seq(seq_pl: pl.DataFrame):
    if isinstance(seq_pl, pl.LazyFrame): 
        seq_pl = seq_pl.collect()
    df = seq_pl.to_pandas()
    X = df[feat_cols].to_numpy(dtype=np.float32)
    X = scaler.transform(X)
    X = np.nan_to_num(X, nan=0.0, posinf=0.0, neginf=0.0).astype(np.float32, copy=False)
    L = min(X.shape[0], pad_len)
    X = pad_to_len(X, pad_len, "post", "post")
    xb = torch.from_numpy(X).unsqueeze(0).to(device)
    Lb = torch.tensor([L], dtype=torch.long, device=device)
    return xb, Lb

def predict(sequence: pl.DataFrame, demographics: pl.DataFrame) -> str:
    xb, Lb = _prep_seq(sequence)
    with torch.inference_mode():
        logits = model(xb, Lb)
        idx = int(torch.softmax(logits, -1).argmax(1).item())
    return id2label[idx]

inference_server = kaggle_evaluation.cmi_inference_server.CMIInferenceServer(predict)

In [ ]:
if os.getenv("KAGGLE_IS_COMPETITION_RERUN"):
    inference_server.serve()
else:
    inference_server.run_local_gateway(
        data_paths=(
            f"{COMP_DIR}/test.csv",
            f"{COMP_DIR}/test_demographics.csv",
        )
    )